<a href="https://colab.research.google.com/github/dashang/LangChainInvestmentSplitWithTools/blob/main/%5B14th_Dec%5D_GenAi_with_Tool_Stock_Split_and_buying_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 langchain-google-genai==0.0.6 google-search-results

Found existing installation: langchain 1.1.3
Uninstalling langchain-1.1.3:
  Successfully uninstalled langchain-1.1.3
Found existing installation: langchain-core 1.1.3
Uninstalling langchain-core-1.1.3:
  Successfully uninstalled langchain-core-1.1.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 24.0 MB/s eta 0:00:00

In [8]:

import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import SerpAPIWrapper
from google.colab import userdata

# --- CONFIGURATION ---
# 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')

# 2. SerpAPI Key (Get free at serpapi.com)
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY") or os.environ["GOOGLE_API_KEY"] == "YOUR_GOOGLE_GEMINI_KEY":
    print("⚠️ STOP: Please enter your Google API Key!")
elif not os.environ.get("SERPAPI_API_KEY") or os.environ["SERPAPI_API_KEY"] == "YOUR_SERPAPI_KEY":
    print("⚠️ STOP: Please enter your SerpAPI Key!")
else:
    try:
        # --- DEFINE TOOLS ---
        # 1. Real Web Search
        search = SerpAPIWrapper()

        @tool
        def search_web(query: str):
            """Useful for searching the internet for current events, prices, and weather."""
            print(f"    🔎 [Search Tool] Googling: '{query}'")
            return search.run(query)

        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"

        tools = [search_web, calculator]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        budget = input("Enter your Bugdet to invest in INR currency: ")
        stock1 = input("Enter first stock symbol: ")
        stock2 = input("Enter second stock symbol: ")

        print(f"You entered: {stock1} and {stock2}")
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")

        query = f"My Budget is in Indian Rupees (INR) {budget} and search on internet share prices of {stock1} and {stock2} in BSE (indian Stock Exchange), currency we trade in is INR and show its share price and split the value of budget in equal half so that user can buy stock. Round off value of qty. All currency is in Indian rupees"

        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")


Enter your Bugdet to invest in INR currency: 7000
Enter first stock symbol: hdfcbank
Enter second stock symbol: sbin
You entered: hdfcbank and sbin

--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...
Action: search_web
Action Input: HDFC Bank share price BSE    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE'

Observation: {'type': 'finance_results', 'title': 'News', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:Action: search_web
Action Input: HDFC Bank share price BSE in INR    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE in INR'

Observation: {'type': 'finance_results', 'title': 'Overview', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:Action: search_web
Action Input: HDFC Bank share price BSE today INR    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE today INR'

Observation: {'type': 'finance_results', 'title': 'People also ask', 'exchange': 'BOM', 'stock': '500180', 'cu

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 5.066065319s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5
}
, retry_delay {
  seconds: 5
}
].


Action: search_web
Action Input: HDFC Bank share price BSE in Indian Rupees    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE in Indian Rupees'

Observation: {'type': 'finance_results', 'title': 'People also ask', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 2.958886186s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5
}
, retry_delay {
  seconds: 2
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 58.849328023s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/g

Action: search_web
Action Input: HDFC Bank share price BSE today in INR    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE today in INR'

Observation: {'type': 'finance_results', 'title': 'People also ask', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:Action: search_web
Action Input: HDFC Bank share price BSE today in INR    🔎 [Search Tool] Googling: 'HDFC Bank share price BSE today in INR'

Observation: {'type': 'finance_results', 'title': 'People also ask', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:Action: search_web
Action Input: HDFC Bank share price today BSE in INR    🔎 [Search Tool] Googling: 'HDFC Bank share price today BSE in INR'

Observation: {'type': 'finance_results', 'title': 'People also ask', 'exchange': 'BOM', 'stock': '500180', 'currency': 'USD', 'price': 35.89}
Thought:Action: search_web
Action Input: HDFC Bank share price BSE India today    🔎 [Search Tool] Googling: 'HDFC Bank share pric